In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import seaborn as sn
import statsmodels.formula.api as smf
from IPython.display import Image

plt.style.use('ggplot')

In [19]:
# Connect to Oracle
eng = nivapy.da.connect()

Connection successful.


# MARTINI - Process Norwegian river data

This notebook generates a dataset of riverine inputs to Skagerrak from 23 major Norwegian rivers. It builds upon the data exploration originally described in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/water_chem.ipynb). The aim is to generate a complete dataset of daily nutrient concentrations covering the period from 2015 to 2017 inclusive. The following parameters are of interest: NH4, NO3, TON, Tot-N, SRP, TOP, Tot-P, DOC and TOC. These basic quantities can then be further subdived according to the requirements of the marine model.

I have already extracted water quality data for as many rivers as possible from Vannmiljø. Based on the preliminary data exploration described in the [previous notebook](http://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/water_chem.ipynb) think the following workflow seems like the best strategy:

 1. **Concentration-discharge relationships for each site**
 
 Regress Tot-N, Tot-P and TOC against flow for each site, using all the data available between 2000 and 2018. Use these relationships to estimate complete daily time series for 2015 to 2017.  
 
 2. **Concentration-concentration relationships for the dataset as a whole**
 
 Many of the sites do not have enough data for NH4, NO3, SRP or DOC to be able to derive C-Q relationships directly. Instead, I will pool all the data and estimate "overall" regressions between the following parameters:
  
  * NH4 vs. Tot-N
  * NO3 vs. Tot-N
  * SRP vs. Tot-P
  * DOC vs. TOC <br><br>
  
 Time series for these varaiables can then be estimated from the Tot-N, Tot-P and TOC series calculated in step 1. Note that the [exploratory notebook](http://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/water_chem.ipynb) indicates this will work OK for NO3 and DOC, but results for NH4 and SRP will likely be poor.

 3. **Estimate other parameters of interest**
 
 Additional model parameters must be estimated from the monthly series:
 
  * Calculate organic N: $TON = TotN - NH4 - NO3$
  * Decide on what to do about other P species. The best approach probably depends on how organic P is treated in the model (i.e. whether it is assumed to be readily available or not) <br><br> 
 
## 1. List Norwegian rivers

In [3]:
# Read Andre's list of rivers
riv_xlsx = r'../../../andre_river_data/andre_martini_rivers_jes.xlsx'
riv_df = pd.read_excel(riv_xlsx, sheet_name='rivers')

# Filter to Norway
riv_df = riv_df.loc[riv_df['martini_code'].str.startswith('n')]

riv_df

,martini_id,station_name,resa_id,vannmiljo_id,lat,lon,martini_code
9,7,Haldenvassdraget,29830.0,001-38541,59.1187,11.3697,nmca001
10,9,Glomma-Vesterelva,29617.0,002-38516,59.1846,10.8819,nmca002
11,8,Glomma-Østerelva,29617.0,002-38516,59.1813,10.9527,nmca002
12,10,Mossevassdraget,30129.0,003-30718,59.4392,10.6610,nmca003
13,11,Hølenelva,30021.0,004-60940,59.5278,10.6900,nmca004
14,12,Årungelva,30023.0,005-38775,59.7223,10.7280,nmca005
15,13,Akerselva,30078.0,006-51732,59.9048,10.7538,nmca006
16,14,Lysakerelva,30082.0,007-56495,59.9115,10.6419,nmca007
17,15,Sandvikselva,30022.0,008-60878,59.8874,10.5288,nmca008
18,16,Årosvassdraget,30025.0,009-29296,59.7040,10.5201,nmca009


## 2. Tidy/aggregate chemistry data

The code below repeats some processing from the previous notebook to create a tidy dataset for the subsequent analysis.

In [4]:
# Read tidied Vannmiljø data back from csv for speed
vm_csv = r'../../../vannmiljo_export/vannmiljo_n_p_c_martini.csv'
wc_df = pd.read_csv(vm_csv)

# Convert dates ignoring times
wc_df['sample_date'] = pd.to_datetime(wc_df['sample_date']).dt.date

# Remove unusual pars with v. limited data
wc_df = wc_df.query('parameter not in ("N-SNOX", "P-TOT-F", "PON")')

# Aggregate soluble P params as SRP
srp_pars = ['P-PO4-F', 'P-PO4', 'P-ORTO-F', 'P-ORTO']
wc_df['parameter'] = np.where(wc_df['parameter'].isin(srp_pars), 'SRP', wc_df['parameter'])

# Merge cols
wc_df['par_unit'] = wc_df['parameter'] + '_' + wc_df['unit']
del wc_df['depth1'], wc_df['depth2'], wc_df['parameter']
del wc_df['flag'], wc_df['unit']

# Average duplicates
idx_cols = ['vassnr', 'station_name', 'lon', 'lat', 'sample_date', 'par_unit']
wc_df = wc_df.groupby(idx_cols).mean().reset_index()
wc_df.head()

,vassnr,station_name,lon,lat,sample_date,par_unit,value
0,1,"Tista, utløp Femsjøen (FEMU)",11.444334,59.127828,2000-01-01,N-NH4_µg/l N,21.0
1,1,"Tista, utløp Femsjøen (FEMU)",11.444334,59.127828,2000-01-01,N-NO3_µg/l N,574.0
2,1,"Tista, utløp Femsjøen (FEMU)",11.444334,59.127828,2000-01-01,N-TOT_µg/l N,936.0
3,1,"Tista, utløp Femsjøen (FEMU)",11.444334,59.127828,2000-01-01,SRP_µg/l P,1.0
4,1,"Tista, utløp Femsjøen (FEMU)",11.444334,59.127828,2000-01-01,TOC_mg/l C,7.1


## 3. Concentration-discharge relationships

### 3.1. Regression relationships

Section 2.1.5 of the [exploratory notebook](http://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/water_chem.ipynb) produced C-Q regression plots for each site. The code below calculates OLS regression equations and saves the coefficients for later use.

**Note:** The relationship between C and Q is typically non-linear, and it is usual to fit a model of the following form

$$ln(C) = \beta_0 + \beta_1 ln(Q) + \epsilon_i$$

In order to make predictions for $C$, it is therefore necessary to back-transform the regression. This incurs **back-transformation bias**, which must be corrceted. One way to do this to use Duan's "smearing estimator"

$$E[y|x] = \alpha * exp(x´ \hat \beta)$$

where 

$$\alpha = \frac{1}{N} \sum_i{exp(\hat \epsilon_i)}$$

and the $\hat \epsilon_i$ are the regression residuals.

This approach assumes that errors are **heteroscedastic**, but not necessarily Gaussian. The code below creates diagnotic plots of the residuals, so this can be assessed.

In [5]:
# Simple OLS regression against flow
reg_dict = {}
for vass in wc_df['vassnr'].unique():
    # Skip sites with no data
    if vass not in (5, 10):
        # Get stn data
        df = wc_df.query('vassnr == @vass')
        
        # Set values equal to 0 to NaN (zeros cause log issues, and no lab 
        # method can actually report 0)
        df[df==0] = np.nan

        # Get RESA stn ID
        mar_cd = 'nmca%03d' % vass
        resa_id = riv_df.query('martini_code == @mar_cd')['resa_id'].iloc[0]

        # Get flow data from NIVABASE
        q_df = nivapy.da.extract_resa_discharge(resa_id,
                                                '2000-01-01',
                                                '2017-12-31',
                                                eng,
                                                plot=False)

        # Restructure chem
        df = df.sort_values(['sample_date', 'par_unit'])
        df.set_index(idx_cols, inplace=True)
        df = df.unstack('par_unit').reset_index().sort_values('sample_date')
        df.index = df['sample_date']
        del df['vassnr'], df['station_name'], df['lon'], df['lat'], df['sample_date']
        df.columns = df.columns.get_level_values(1)

        # Get list of chem cols
        if vass == 4:
            # No TOC
            chem_cols = ['N-TOT_µg/l N', 'P-TOT_µg/l P']
        else:
            chem_cols = ['N-TOT_µg/l N', 'P-TOT_µg/l P', 'TOC_mg/l C']

        # Join to Q
        df = df.join(q_df, how='left')
        
        # Get logged data
        df2 = np.log(df)
        df2.columns = ['log(%s)' % i for i in df2.columns]
        df = df.join(df2)

        # Regression
        for col in chem_cols:
            # OLS regression
            res = smf.ols(formula='Q("log(%s)") ~ Q("log(flow_m3/s)")' % col, data=df).fit()

            # Plot diagnotsics
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
            
            axes[0].plot(res.model.exog[:, 1], res.resid, 'ro')
            axes[0].set_xlabel('log[Flow (m3/s)]')
            axes[0].set_ylabel('Residual')
            
            sn.distplot(res.resid.values, ax=axes[1])
            axes[1].set_xlabel('Residual')
            
            plt.suptitle('%s at vassområde %03d' % (col.split('_')[0], vass))
            plt.tight_layout()
            
            # Save png
            png_path = r'../plots/flow_conc_reg_resid/vassnr_%03d_%s.png' % (vass, col.split('_')[0])
            plt.savefig(png_path, dpi=300)
            plt.close()  
        
            # Add to results
            reg_dict[(vass, col)] = res

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)
/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which w

### 3.2. Calculate daily Tot-N, Tot-P and TOC

The code below uses the equations from Section 3.1 to estimate daily Tot-N, Tot-P and TOC from discharge.

**Note:** On days where measured values are available, these are used in preference to the calculated values. This produces some sudden spikes in the data, which is an indication that the regression equations aren't very good in many cases.

In [20]:
# Loop over sites
data_dict = {}
for vass in wc_df['vassnr'].unique():
    # Skip sites with no data
    if vass not in (5, 10):
        # Get stn data
        df = wc_df.query('vassnr == @vass')
        
        # Set values equal to 0 to NaN (zeros cause log issues, and no lab 
        # method can actually report 0)
        df[df==0] = np.nan

        # Get RESA stn ID
        mar_cd = 'nmca%03d' % vass
        resa_id = riv_df.query('martini_code == @mar_cd')['resa_id'].iloc[0]

        # Get flow data from NIVABASE
        q_df = nivapy.da.extract_resa_discharge(resa_id,
                                                '2015-01-01',
                                                '2017-12-31',
                                                eng,
                                                plot=False)
        q_df = q_df.resample('D').mean()
        
        # Remove zeros and then interpolate again
        q_df[q_df==0] = np.nan
        q_df.interpolate(kind='linear', inplace=True)

        # Restructure chem
        df = df.sort_values(['sample_date', 'par_unit'])
        df.set_index(idx_cols, inplace=True)
        df = df.unstack('par_unit').reset_index().sort_values('sample_date')
        df.index = df['sample_date']
        del df['vassnr'], df['station_name'], df['lon'], df['lat'], df['sample_date']
        df.columns = df.columns.get_level_values(1)

        # Get list of chem cols
        if vass == 4:
            # No TOC
            chem_cols = ['N-TOT_µg/l N', 'P-TOT_µg/l P']
        else:
            chem_cols = ['N-TOT_µg/l N', 'P-TOT_µg/l P', 'TOC_mg/l C']
            
        # Get just cols of interest
        df = df[chem_cols]

        # Join to Q
        df = q_df.join(df, how='left')
        
        # Loop over chem data
        for col in chem_cols:
            # Get regression result
            res = reg_dict[(vass, col)]
            
            # Calc log(C)
            ln_c = res.params[0] + res.params[1]*np.log(df['flow_m3/s'])

            # Back-transform
            alpha = (np.exp(res.resid.values)).mean()
            concs = alpha*np.exp(ln_c)
            
            # Patch missing in main series
            df[col] = df[col].fillna(concs)
            
            # Add to results
            data_dict[vass] = df

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


## 4. Concentration-concentration relationships

### 4.1. Regression relationships

The tables below show summary statistics for each regression. Note that **I've removed the intercept term** (unlike on the rough plots in the [exploratory notebook](http://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/water_chem.ipynb)), as it seems sensible to fix the intercept at the origin for all these models.

In [7]:
# Restructure chem
df = wc_df.copy()
df = df.sort_values(['vassnr', 'sample_date', 'par_unit'])
df.set_index(idx_cols, inplace=True)
df = df.unstack('par_unit').reset_index().sort_values(['vassnr', 'sample_date'])
del df['station_name'], df['lon'], df['lat']
df.columns = df.columns.get_level_values(1)
cols = list(df.columns)
cols[:2] = ['vassnr', 'sample_date']
df.columns = cols

# Filter obviously contaminated P sample
df = df[df['P-TOT_µg/l P']<3000]

### 4.1.1. NH4 vs. Tot-N

$R^2 \approx 38\%$

$p \approx 0$

$NH_4 \approx 0.039*TotN$

In [8]:
# NH4
nh4_reg = smf.ols(formula='Q("N-NH4_µg/l N") ~ Q("N-TOT_µg/l N") - 1', data=df).fit()
print(nh4_reg.summary())

                            OLS Regression Results                            
Dep. Variable:      Q("N-NH4_µg/l N")   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     1763.
Date:                Fri, 14 Dec 2018   Prob (F-statistic):          2.98e-302
Time:                        16:05:13   Log-Likelihood:                -14141.
No. Observations:                2942   AIC:                         2.828e+04
Df Residuals:                    2941   BIC:                         2.829e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Q("N-TOT_µg/l N")     0.0386      0.00

### 4.1.2. NO3 vs. Tot-N

$R^2 \approx 94\%$

$p \approx 0$

$NO_3 \approx 0.56*TotN$

In [9]:
# NO3
no3_reg = smf.ols(formula='Q("N-NO3_µg/l N") ~ Q("N-TOT_µg/l N") - 1', data=df).fit()
print(no3_reg.summary())

                            OLS Regression Results                            
Dep. Variable:      Q("N-NO3_µg/l N")   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                 2.138e+04
Date:                Fri, 14 Dec 2018   Prob (F-statistic):               0.00
Time:                        16:05:19   Log-Likelihood:                -7588.7
No. Observations:                1363   AIC:                         1.518e+04
Df Residuals:                    1362   BIC:                         1.518e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Q("N-TOT_µg/l N")     0.5591      0.00

### 4.1.3. SRP vs. Tot-P

$R^2 \approx 48\%$

$p \approx 0$

$SRP \approx 0.27*TotP$

In [10]:
srp_reg = smf.ols(formula='Q("SRP_µg/l P") ~ Q("P-TOT_µg/l P") - 1', data=df).fit()
print(srp_reg.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("SRP_µg/l P")   R-squared:                       0.484
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     2974.
Date:                Fri, 14 Dec 2018   Prob (F-statistic):               0.00
Time:                        16:05:23   Log-Likelihood:                -11325.
No. Observations:                3177   AIC:                         2.265e+04
Df Residuals:                    3176   BIC:                         2.266e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Q("P-TOT_µg/l P")     0.2736      0.00

### 4.1.4. DOC vs. TOC

$R^2 \approx 100\%$

$p \approx 0$

$DOC \approx 0.97*TOC$

In [11]:
doc_reg = smf.ols(formula='Q("DOC_mg/l C") ~ Q("TOC_mg/l C") - 1', data=df).fit()
print(doc_reg.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("DOC_mg/l C")   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 8.949e+04
Date:                Fri, 14 Dec 2018   Prob (F-statistic):          1.49e-181
Time:                        16:05:28   Log-Likelihood:                 71.539
No. Observations:                 127   AIC:                            -141.1
Df Residuals:                     126   BIC:                            -138.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Q("TOC_mg/l C")     0.9651      0.003    2

### 4.2. Estimate NH4, NO3, SRP and DOC

In [21]:
# Loop over sites
for vass in data_dict.keys():
    # Get data
    df = data_dict[vass]
    
    # Calc params
    if 'N-TOT_µg/l N' in df.columns:
        df['N-NH4_µg/l N'] = nh4_reg.params[0]*df['N-TOT_µg/l N']
        df['N-NO3_µg/l N'] = no3_reg.params[0]*df['N-TOT_µg/l N']
        
    if 'P-TOT_µg/l P' in df.columns:
        df['SRP_µg/l P'] = srp_reg.params[0]*df['P-TOT_µg/l P']
        
    if 'TOC_mg/l C' in df.columns:
        df['DOC_mg/l C'] = doc_reg.params[0]*df['TOC_mg/l C']
        
    # Update results
    data_dict[vass] = df

## 5. Estimate other parameters of interest

In [22]:
# Loop over sites
df_list = []
for vass in data_dict.keys():
    # Get data
    df = data_dict[vass]
    
    # Calc. TON
    df['N-TON_µg/l N'] = df['N-TOT_µg/l N'] - df['N-NH4_µg/l N'] - df['N-NO3_µg/l N']
    
    # Plot
    df.plot(subplots=True, 
            layout=(3,3), 
            figsize=(15,10),
            legend=False,
            title=list(df.columns))
    
    plt.tight_layout()

    # Save png
    png_path = r'../plots/daily_series/vassnr_%03d.png' % vass
    plt.savefig(png_path, dpi=300)
    plt.close() 
    
    # Tidy
    cols = list(df.columns)
    df.reset_index(inplace=True)
    df['vassnr'] = vass
    df = df[['vassnr', 'date'] + cols]
    
    # Add to output
    df_list.append(df)
    
# Combine
df = pd.concat(df_list, axis=0)

df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,DOC_mg/l C,N-NH4_µg/l N,N-NO3_µg/l N,N-TON_µg/l N,N-TOT_µg/l N,P-TOT_µg/l P,SRP_µg/l P,TOC_mg/l C,date,flow_m3/s,vassnr
0,8.188129,32.441643,469.346466,337.718565,839.506674,16.311650,4.463461,8.484315,2015-01-01,23.146906,1
1,8.257358,32.930202,476.414647,342.804479,852.149328,16.746516,4.582456,8.556048,2015-01-02,40.579857,1
2,8.251991,32.892210,475.865009,342.408987,851.166206,16.712523,4.573154,8.550486,2015-01-03,38.858033,1
3,8.221613,32.677544,472.759337,340.174299,845.611180,16.521008,4.520749,8.519009,2015-01-04,30.386127,1
4,8.782321,30.914959,447.259307,321.825735,800.000000,25.000000,6.840910,9.100000,2015-01-05,28.937313,1


## 4. Patched monthly records for Tot-N, Tot-P and TOC

The code below is adapted from [Section 2.1.7 of the previous notebook](http://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/water_chem.ipynb#2.1.7.-Fill-gaps-with-long-term-monthly-means). It creates continuous monthly series for Tot-N, Tot-P and TOC (where available) at each site by patching with long-term averages, interpolating and back-filling.

In [ ]:
# Container for results
patch_dict = {}

# Loop over cacthments
for vass in wc_df['vassnr'].unique():
    # Get stn data
    df = wc_df.query('vassnr == @vass')

    # Restructure chem
    df = df.sort_values(['sample_date', 'par_unit'])
    df.set_index(idx_cols, inplace=True)
    df = df.unstack('par_unit').reset_index().sort_values('sample_date')
    df.index = df['sample_date']
    del df['vassnr'], df['station_name'], df['lon'], df['lat'], df['sample_date']
    df.columns = df.columns.get_level_values(1)
    
    # Get just cols of interest
    col_int = set(['TOC_mg/l C', 'N-TOT_µg/l N', 'P-TOT_µg/l P'])
    cols = list(col_int.intersection(set(df.columns)))
    df = df[cols]
    
    # Long-term monthly means
    df2 = df.groupby(df.index.month).mean()
    
    # Get series from 2015 to 2017 with monthly avgs. where available
    dates_df = pd.DataFrame(index=pd.date_range(start='2015-01-01',
                                                end='2017-12-31',
                                                freq='M'))
    df.dropna(inplace=True)
    df = df.resample('M').mean()
    df = dates_df.join(df, how='left')
    
    # Iterate over rows filling NaN
    cols = df.columns
    for idx, row in df.iterrows():
        # Get the month for this row
        month = idx.month
        
        # Loop over params.
        for col in cols:
            # If NaN, fill with long-term mean, if available
            if pd.isnull(row[col]):
                try:
                    df.loc[idx][col] = df2.loc[month][col]
                except KeyError:
                    # No data at all for this month between 2010 and 2018
                    # Leave as NaN
                    pass
    
    # Fill remaining NaNs with linear interp.
    df.interpolate(kind='linear', inplace=True)
    
    # Backfill any final NaNs at start of series
    df.fillna(method='backfill', inplace=True)

    # Add to output
    patch_dict[vass] = df

In [ ]:
# Show example of monthly patched series for vassnr = 1
ax = patch_dict[1].plot(subplots=True, figsize=(15,6))

## 5. Estimate monthly NH4, NO3, SRP & DOC and patch using regression relationships

In [ ]:
# Loop over cacthments
for vass in wc_df['vassnr'].unique():
    # Get stn data
    df = wc_df.query('vassnr == @vass')

    # Restructure chem
    df = df.sort_values(['sample_date', 'par_unit'])
    df.set_index(idx_cols, inplace=True)
    df = df.unstack('par_unit').reset_index().sort_values('sample_date')
    df.index = df['sample_date']
    del df['vassnr'], df['station_name'], df['lon'], df['lat'], df['sample_date']
    df.columns = df.columns.get_level_values(1)
    
    # Get just cols of interest
    col_int = set(['N-NH4_µg/l N', 'N-NO3_µg/l N', 'SRP_µg/l P', 'DOC_mg/l C'])
    cols = list(col_int.intersection(set(df.columns)))
    df = df[cols]
    
    # Get series from 2015 to 2017 with monthly avgs. where available
    dates_df = pd.DataFrame(index=pd.date_range(start='2015-01-01',
                                                end='2017-12-31',
                                                freq='M'))
    df.dropna(inplace=True)
    df = df.resample('M').mean()
    df = dates_df.join(df, how='left')
    
    # Join "Total" data series
    df = df.join(patch_dict[vass])
    print(vass)
    # Patch series
    for col in cols:
        if col == 'SRP_µg/l P':
            df[col] = df[col].fillna(reg_dict[col]*df['P-TOT_µg/l P'])
        elif col == 'DOC_mg/l C':
            df[col] = df[col].fillna(reg_dict[col]*df['TOC_mg/l C'])
        else:
            df[col] = df[col].fillna(reg_dict[col]*df['N-TOT_µg/l N'])
df    